In [153]:
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option("display.width", 300)

In [154]:
# Load csv file into dataframe
books = pd.read_csv("../data/03_Library Systembook.csv")

# Drop items with blank Id or CustomerID values
books = books.dropna(subset=['Id', "Customer ID"])
print(books)

      Id                                     Books Book checkout Book Returned Days allowed to borrow  Customer ID
0    1.0                       Catcher in the Rye   "20/02/2023"    25/02/2023                2 weeks          1.0
1    2.0          Lord of the rings the two towers  "24/03/2023"    21/03/2023                2 weeks          2.0
2    3.0  Lord of the rings the return of the kind  "29/03/2023"    25/03/2023                2 weeks          3.0
3    4.0                                The hobbit  "02/04/2023"    25/03/2023                2 weeks          4.0
4    5.0                                     Dune   "02/04/2023"    25/03/2023                2 weeks          5.0
5    6.0                              Little Women  "02/04/2023"    01/05/2023                2 weeks          1.0
6    7.0                                        IT  "10/04/2063"    03/04/2023                2 weeks          6.0
7    8.0                                   Misery   "15/04/2023"    03/04/2023  

In [155]:
books = books.set_index("Id")

In [156]:
# Reformat book checkout column - parse date contained in "" then return as datetime object
def format_book_checkout(date_str: str):
    try:
        dt = datetime.strptime(date_str, '"%d/%m/%Y"')
        return dt
    except ValueError:
        return
books['Book checkout'] = books.apply(lambda row: format_book_checkout(row['Book checkout']), axis=1)

# Remove rows with invalid checkout dates
books = books[~books['Book checkout'].isnull()]

# Convert book returned column to datetime
books['Book Returned'] = pd.to_datetime(books['Book Returned'], format="%d/%m/%Y")

# Add calculated column - loaned days
books['DaysOnLoan'] = (books['Book Returned'] - books['Book checkout'])/np.timedelta64(1, "D")

In [157]:
# Replace NA values with None
books = books.replace({np.nan: None})

# Show df
print(books.dtypes)
print(books)

Books                             object
Book checkout             datetime64[ns]
Book Returned             datetime64[ns]
Days allowed to borrow            object
Customer ID                      float64
DaysOnLoan                       float64
dtype: object
                                         Books Book checkout Book Returned Days allowed to borrow  Customer ID  DaysOnLoan
Id                                                                                                                        
1.0                        Catcher in the Rye     2023-02-20    2023-02-25                2 weeks          1.0         5.0
2.0           Lord of the rings the two towers    2023-03-24    2023-03-21                2 weeks          2.0        -3.0
3.0   Lord of the rings the return of the kind    2023-03-29    2023-03-25                2 weeks          3.0        -4.0
4.0                                 The hobbit    2023-04-02    2023-03-25                2 weeks          4.0        -8.0
5.